This notebook downloads Sephora user product reviews. The original csv being read "df" is a listing of Sephora products from downloaded from Kaggle at https://www.kaggle.com/raghadalharbi/all-products-available-on-sephora-website

Due to large number of reviews a product can have, each product with more than 150 reviews is limited to 10% of all reviews.

2 CSVs are exported. 1) CSV of product review statistics ("products_df.csv") and, 2) CSV of individual reviews ("reviews.csv")

Another CSV is exported for standardizing product ID ("product_id_correction.csv")

In [1]:
import pandas as pd
import requests
import json
from tqdm.notebook import tqdm

In [2]:
pd.options.display.max_columns = 50

In [3]:
df = pd.read_csv('/Users/admin/Downloads/facial_products.csv')

In [4]:
master = pd.DataFrame()

In [5]:
review = pd.DataFrame()
products_df = pd.DataFrame()

In [7]:
file = '/Users/admin/Desktop/Uchicago/Data Platform Engineering/Project/reviews2.csv'
products = '/Users/admin/Desktop/Uchicago/Data Platform Engineering/Project/products2.csv'
calls = 0
prod_cols=['Description', 'Id', 'ImageUrl', 'Name', 'ReviewStatistics','TotalReviewCount']
review_cols=['UserNickname','Rating','ReviewText','ContextDataValues','ProductId','AuthorId','SubmissionTime','IsRecommended','ContentLocale','Id']
downloaded = pd.read_csv(products)['Id'].to_list()
for k, v in tqdm(df.iterrows(), total = len(df)):
    continue
    offset = 0
    limit = 50
    temp = pd.DataFrame()
    pid = v['URL'].split('grid:')[1].upper()
    if pid in downloaded:
        continue
    while True:
        url = f'https://api.bazaarvoice.com/data/reviews.json?Filter=contentlocale%3Aen*&Filter=ProductId%3A{pid}&Sort=SubmissionTime%3Adesc&Limit={limit}&Offset={offset}&Include=Products%2CComments&Stats=Reviews&passkey=caQ0pQXZTqFVYA1yYnnJ9emgUiW59DXA85Kxry8Ma02HE&apiversion=5.4&Locale=en_US'
        r = requests.get(url,timeout=10)
        calls += 1
        response = json.loads(r.text)
        total = response['TotalResults']
        if total == 0:
            break

        temp = pd.concat([temp, pd.DataFrame(response['Results'])[review_cols]])
        offset += limit

        c = len(temp)
        if c >= total or (c >= 100 and c > int(total * 0.1)):
            break
    if total == 0:
        continue
    if len(temp) > 0:
        review = pd.concat([review,temp])
    
    products_df = pd.concat([products_df, pd.DataFrame(response['Includes']['Products']).T.reset_index()])
    products_df.reset_index(inplace=True, drop=True)
    
    if k % 1000 == 0:
        products_df[prod_cols].to_csv(products, index=False)
        review.to_csv(file, index=False)

products_df[prod_cols].to_csv(products, index=False)
review.to_csv(file, index=False)        
        

  0%|          | 0/4419 [00:00<?, ?it/s]

In [349]:
df['pid'] = df['URL'].str.split('grid:').str[1].str.upper()

In [352]:
df[['id', 'pid']].to_csv('product_id_correction.csv', index = False)